In [10]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px

import boto3
from io import StringIO

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=r'C:\Users\ambri\Desktop\vscodetraining\Training\FullStack\.env')

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Exploratory Data Analysis to find the top 20 hotels in the top 5 cities

In [11]:
#Read the csv that we create in the scrap notebook
df = pd.read_csv("Kayak_Project_scrap_booking.csv")
df.head()

,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,Carcassonne,43.206183545263,2.361151027302,Studio pour 2 avec terrasse au pied de la Cité...,"9,5",174 expériences vécues,"36 Rue Longue, 11000 Carcassonne, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Studio pour 2 avec terrasse au p...,https://www.booking.com/hotel/fr/studio-pour-2...
1,Carcassonne,43.2114588,2.3508581,Marius - studio au cœur de la bastide,"8,0",112 expériences vécues,"9 Ruelle Perrot, 11000 Carcassonne, France","Connexion Wi-Fi gratuite,Chambres non-fumeurs,...",L’hébergement Marius - studio au cœur de la ba...,https://www.booking.com/hotel/fr/marius-studio...
2,Aigues Mortes,43.5672196715711,4.189343601465225,Hôtel Saint Louis,"8,5",1 577 expériences vécues,"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F...","Parking,Spa et centre de bien-être,Connexion W...","Situé dans le cœur historique d'Aigues-Mortes,...",https://www.booking.com/hotel/fr/saint-louis-a...
3,Carcassonne,43.2140544,2.3460563,Chez Pauline & Vincent Côté Canal,"9,8",11 expériences vécues,"19 Avenue Pierre Charles Lespinasse, 11000 Car...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Chez Pauline & Vincent Côté Cana...,https://www.booking.com/hotel/fr/cote-canal-ca...
4,Carcassonne,43.2082373962379,2.36704409122467,La Rapière,"8,4",3 099 expériences vécues,"9 Montée Gaston Combéléran, 11000 Carcassonne,...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",La chambre d'hôtes La Rapiere est située à Car...,https://www.booking.com/hotel/fr/la-rapia-re.f...


In [12]:
# Types of columns
df.dtypes

city                object
hotel_lat           object
hotel_lon           object
hotel_name          object
hotel_review        object
hotel_nbr_review    object
hotel_address       object
hotel_facilities    object
hotel_desc          object
hotel_url           object
dtype: object

In [13]:
# Convert float data
df['hotel_lat'] = pd.to_numeric(df['hotel_lat'], errors='coerce')
df['hotel_lon'] = pd.to_numeric(df['hotel_lon'], errors='coerce')

In [14]:
#Clean the str to have only float
df['hotel_nbr_review'] = df['hotel_nbr_review'].str.extract(r'([\d.]+)')
df['hotel_nbr_review'] = pd.to_numeric(df['hotel_nbr_review'])

#Standardization of the object then transform to numeric
df['hotel_review'] = df['hotel_review'].str.replace(',', '.')
df['hotel_review'] = pd.to_numeric(df['hotel_review'], errors='coerce')

# Load the dataframe in a data lake : S3 bucket
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY"], 
                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket="citiesinfoskayakjedha")

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = df.to_csv()
put_object = bucket.put_object(Key="all_hotel_info.csv", Body=csv)

df.head()

,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,Carcassonne,43.206184,2.361151,Studio pour 2 avec terrasse au pied de la Cité...,9.5,174.0,"36 Rue Longue, 11000 Carcassonne, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Studio pour 2 avec terrasse au p...,https://www.booking.com/hotel/fr/studio-pour-2...
1,Carcassonne,43.211459,2.350858,Marius - studio au cœur de la bastide,8.0,112.0,"9 Ruelle Perrot, 11000 Carcassonne, France","Connexion Wi-Fi gratuite,Chambres non-fumeurs,...",L’hébergement Marius - studio au cœur de la ba...,https://www.booking.com/hotel/fr/marius-studio...
2,Aigues Mortes,43.567220,4.189344,Hôtel Saint Louis,8.5,1.0,"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F...","Parking,Spa et centre de bien-être,Connexion W...","Situé dans le cœur historique d'Aigues-Mortes,...",https://www.booking.com/hotel/fr/saint-louis-a...
3,Carcassonne,43.214054,2.346056,Chez Pauline & Vincent Côté Canal,9.8,11.0,"19 Avenue Pierre Charles Lespinasse, 11000 Car...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Chez Pauline & Vincent Côté Cana...,https://www.booking.com/hotel/fr/cote-canal-ca...
4,Carcassonne,43.208237,2.367044,La Rapière,8.4,3.0,"9 Montée Gaston Combéléran, 11000 Carcassonne,...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",La chambre d'hôtes La Rapiere est située à Car...,https://www.booking.com/hotel/fr/la-rapia-re.f...


In [15]:
# Initialization of the SQL engine
USERNAME = "postgres"
HOSTNAME = os.environ["HOSTNAME"]
PASSWORD = os.environ["PASSWORD"]

engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

# Storing the dataframe as SQL tables
df.to_sql('hotel_info', con=engine, if_exists='replace', index=False)

OperationalError: (psycopg2.OperationalError) could not translate host name "database-2.c368u40ewjg3.eu-west-3.rds.amazonaws.com" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [16]:
#Apply a mask with the conditions needed : the review and the nbr of review
conditions = (df['hotel_review']>8.7) & (df['hotel_nbr_review'] > 30)
top_hotels = df[conditions]
top20_hotels = top_hotels.sort_values('hotel_review', ascending=False).head(20).reset_index(drop=True)
top20_hotels

,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,Cassis,43.217600,5.532350,LE SEPT charmant studio aux portes des calanques,9.9,89.0,"7 Avenue des Carriers, 13260 Cassis, France","Piscine extérieure,Parking gratuit,Connexion W...","Situé à Cassis et offrant une vue sur la mer, ...",https://www.booking.com/hotel/fr/le-sept-charm...
1,Cassis,43.215246,5.544668,Chambre d'hôtes Clos du Petit Jésus,9.9,117.0,"4 Av. de Provence, 13260 Cassis, France","Piscine extérieure,Parking gratuit,Connexion W...","Situé à Cassis, à seulement 5 minutes à pied d...",https://www.booking.com/hotel/fr/clos-du-petit...
2,Cassis,43.215246,5.544668,Chambre d'hôtes Clos du Petit Jésus,9.9,117.0,"4 Av. de Provence, 13260 Cassis, France","Piscine extérieure,Parking gratuit,Connexion W...","Situé à Cassis, à seulement 5 minutes à pied d...",https://www.booking.com/hotel/fr/clos-du-petit...
3,Aigues Mortes,43.581709,4.208536,Séjour atypique et insolite sur notre péniche ...,9.9,38.0,"la Coradine Chemin de la Pinède, 30220 Aigues-...","Parking gratuit,Connexion Wi-Fi rapide gratuit...","Situé à Aigues-Mortes, l’hébergement Séjour at...",https://www.booking.com/hotel/fr/peniche-aigue...
4,Avignon,43.945188,4.799626,Apartment 40m2-Air conditioning-Elevator-Under...,9.8,53.0,"3 Rue Velouterie, 84000 Avignon, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Apartment 40m2-Air conditioning-...,https://www.booking.com/hotel/fr/appart-parkin...
5,Avignon,43.945188,4.799626,Apartment 40m2-Air conditioning-Elevator-Under...,9.8,53.0,"3 Rue Velouterie, 84000 Avignon, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Apartment 40m2-Air conditioning-...,https://www.booking.com/hotel/fr/appart-parkin...
6,Avignon,43.945188,4.799626,Apartment 40m2-Air conditioning-Elevator-Under...,9.8,54.0,"3 Rue Velouterie, 84000 Avignon, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Apartment 40m2-Air conditioning-...,https://www.booking.com/hotel/fr/appart-parkin...
7,Avignon,43.945188,4.799626,Apartment 40m2-Air conditioning-Elevator-Under...,9.8,53.0,"3 Rue Velouterie, 84000 Avignon, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Apartment 40m2-Air conditioning-...,https://www.booking.com/hotel/fr/appart-parkin...
8,Avignon,43.950019,4.809769,"Avignon Intra-muros, vue Palais des Papes",9.8,35.0,"7 Rue Saluces, 84000 Avignon, France","Parking,Connexion Wi-Fi gratuite,Chambres fami...",Offrant une vue sur la ville et doté d’une con...,https://www.booking.com/hotel/fr/avignon-intra...
9,Avignon,43.945188,4.799626,Apartment 40m2-Air conditioning-Elevator-Under...,9.8,53.0,"3 Rue Velouterie, 84000 Avignon, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Apartment 40m2-Air conditioning-...,https://www.booking.com/hotel/fr/appart-parkin...


In [17]:
# Plot the map of the top 20 hotels in the top 5 places
fig = px.scatter_map(top20_hotels, lat="hotel_lat",
                        lon="hotel_lon",
                        color='hotel_review',
                        size='hotel_nbr_review',
                        size_max=35,  
                        title='Top 20 hotels in the top 5 places where the weather is best in France in the next 7 days',
                        color_continuous_scale="Viridis",
                        opacity=1,
                        hover_name='hotel_name',
                        width=1200, height=900,  
                        zoom=5)  
fig.update_layout(margin={"r":10,"t":100,"l":10,"b":10})
fig.show()
fig.write_html("top_20_hotels.html")